In [1]:
import csv
import pandas as pd
import spotipy 
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials 
import os
from dotenv import load_dotenv
import time

import requests
import base64
import json
from secrets import *

from pprint import pprint

In [2]:
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")


In [3]:
# Configurer l'authentication
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

seeds = sp.recommendation_genre_seeds()

# Liste des fonctionnalités audio à récupérer pour chaque piste
features = ['nom', 'artistes', 'année', 'popularité', 'durée_ms', 'tonalité', 'mode', 'signature_temps', 'acoustique', 'dansabilité', 'énergie', 'instrumentalité', 'vivacité', 'loudness', 'parlant', 'valence', 'tempo', 'album']

# Créer un nouveau fichier CSV et écrire les en-têtes de colonnes
with open('audio_features_2022.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=features)
    writer.writeheader()

    # Boucler à travers les pistes et récupérer les fonctionnalités audio
    offset = 0
    limit = 50
    total = None
    for s in seeds['genres']:
            for i in range(0,1000,50):
                track_results = sp.search(q=f'genre:{s} year:2022' , type='track', offset=i, limit=50) 
                while total is None or offset < total:
                    # Rechercher des pistes avec l'année 2022 dans le titre de la piste
                    results = sp.search(q=f'genre:{s} year:2022' , type='track', limit=limit, offset=offset)

                    if total is None:
                        total = results['tracks']['total']

                for track in results['tracks']['items']:
                    track_id = track['id']
                    audio_features = sp.audio_features(track_id)[0] if sp.audio_features(track_id) else None
                    
                    # Ignorer la piste si les fonctionnalités audio ne peuvent pas être récupérées
                    if audio_features is None:
                        continue
                    
                    # Formater le nom de l'artiste comme une chaîne de caractères
                    artistes = ', '.join([artiste['name'] for artiste in track['artists']])
                    
                    # Créer un dictionnaire avec les fonctionnalités audio
                    track_features = {
                        'nom': track['name'],
                        'artistes': artistes,
                        'année': track['album']['release_date'][:4], # Extraire l'année de la date de sortie de l'album
                        'popularité': track['popularity'],
                        'durée_ms': audio_features['duration_ms'],
                        'tonalité': audio_features['key'],
                        'mode': audio_features['mode'],
                        'signature_temps': audio_features['time_signature'],
                        'acoustique': audio_features['acousticness'],
                        'dansabilité': audio_features['danceability'],
                        'énergie': audio_features['energy'],
                        'instrumentalité': audio_features['instrumentalness'],
                        'vivacité': audio_features['liveness'],
                        'loudness': audio_features['loudness'],
                        'parlant': audio_features['speechiness'],
                        'valence': audio_features['valence'],
                        'tempo': audio_features['tempo'],
                        'album': track['album']['name'] # Ajouter le nom de l'album à la colonne correspondante
                    }
                    
                    # Écrire les fonctionnalités audio dans le fichier CSV
                    writer.writerow(track_features)

                offset += limit

In [ ]:




# # Step 2 - Use Access Token to call playlist endpoint
# data = {
#     'q' : 'artist:iam ',
#     'type': 'track',
#     'limit': '50',
#     'offset': str(offset),
#     'market' :'US'
# }

# track_url = f"https://api.spotify.com/v1/search"
# headers = {
#     "Authorization": "Bearer " + token
# }

# res = requests.get(url=track_url, headers=headers, data=data)

# print(json.dumps(res.json(), indent=2))

NameError: name 'offset' is not defined

In [ ]:

# Step 1 - Authorization 
url = "https://accounts.spotify.com/api/token"
headers = {}
data = {}

# Encode as Base64
message = f"{client_id}:{client_secret}"
messageBytes = message.encode('ascii')
base64Bytes = base64.b64encode(messageBytes)
base64Message = base64Bytes.decode('ascii')


headers['Authorization'] = f"Basic {base64Message}"
data['grant_type'] = "client_credentials"

r = requests.post(url, headers=headers, data=data)

token = r.json()['access_token']


search_url = f"https://api.spotify.com/v1/search"

headers = {
"Authorization": "Bearer " + token
}

param = {
'q' : 'artist:jul track:mon bijou',
'type': 'track',
'limit': '10',
'market' :'FR'
}

result = requests.get(url=search_url, headers=headers, params=param)
track_items = result.json()['tracks']['items']

pprint(track_items[1]['id'])


'2KDvsfshgJR5Gdfds4SDuJ'


In [ ]:

headers = {
"Authorization": "Bearer " + token
}
id_1=track_items[0]['id']

features_url = f"https://api.spotify.com/v1/audio-features/{id_1}"
data = {'id' : id_1}
resultat = requests.get(url=features_url, headers=headers, params=data)
features_items = resultat.json()

pprint(features_items)


{'acousticness': 0.183,
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/127LVxx0WkxbCcZqp2LAEY',
 'danceability': 0.915,
 'duration_ms': 179781,
 'energy': 0.757,
 'id': '127LVxx0WkxbCcZqp2LAEY',
 'instrumentalness': 0.00026,
 'key': 0,
 'liveness': 0.093,
 'loudness': -8.004,
 'mode': 0,
 'speechiness': 0.143,
 'tempo': 125.026,
 'time_signature': 4,
 'track_href': 'https://api.spotify.com/v1/tracks/127LVxx0WkxbCcZqp2LAEY',
 'type': 'audio_features',
 'uri': 'spotify:track:127LVxx0WkxbCcZqp2LAEY',
 'valence': 0.357}
